<a href="https://colab.research.google.com/github/youngpaper1115/Mobile_Game_Analysis/blob/main/mobilegame_analysis_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from urllib.error import HTTPError

In [2]:
pip install google_play_scraper

     |████████████████████████████████| 52 kB 1.3 MB/s 
  Created wheel for google-play-scraper: filename=google_play_scraper-1.0.3-py3-none-any.whl size=24394 sha256=8f769f97edfd8d91d46fa5957566f4df55c325f49301cacd7139fbc40be1d071
  Stored in directory: /root/.cache/pip/wheels/81/37/0b/4a14be55b449a048cd93d79930b1a980dee7896480defa1923
Successfully built google-play-scraper


In [3]:
import pandas as pd

# for scraping app info and reviews from Google Play
from google_play_scraper import app, Sort, reviews

# for pretty printing data structures
from pprint import pprint

# for storing in MongoDB
import pymongo
from pymongo import MongoClient

# for keeping track of timing
import datetime as dt
from tzlocal import get_localzone

# for building in wait times
import random
import time

In [4]:
## Set up Mongo client
client = MongoClient(host='localhost', port=27017)

## Database for project
app_proj_db = client['app_proj_db']

## Set up new collection within project db for app info
info_collection = app_proj_db['info_collection']

## Set up new collection within project db for app reviews
review_collection = app_proj_db['review_collection']

In [5]:
## Read in file containing app names and IDs
app_df = pd.read_excel('/content/drive/MyDrive/Python_SNU/mobilegame_analysis/df_1.xlsx')
app_df.head()

,Unnamed: 0,idx,title,total_ratings,installs,avg_rating,growth_30days,growth_60days,price,game_cat,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,0,3,Mobile Legends: Bang Bang,30116467,100.0 M,3.72,0.016,0.029,Free,game_action,...,https://m.mobilelegends.com/,mobilelegendsgame@moonton.com,"Oct 31, 2016","Jun 10, 2021",Teen,https://m.mobilelegends.com/en/newsdetail/475,True,True,False,2021-06-16 02:45:58
1,1,4,Brawl Stars,21640230,100.0 M,4.22,0.013,0.058,Free,game_action,...,http://www.supercell.net,gp-info@supercell.com,"Dec 12, 2018","May 20, 2021",Everyone 10+,http://supercell.com/privacy-policy/,False,True,True,2021-06-16 00:44:15
2,2,6,Shadow Fight 2,15069841,100.0 M,4.64,0.007,0.015,Free,game_action,...,http://support.shadowfight2.com,support@nekki.mail.helpshift.com,"Jul 22, 2014","May 19, 2021",Everyone 10+,http://nekki.ru/privacy.php,True,True,False,2021-06-15 21:01:31
3,3,8,Among Us,12989150,100.0 M,3.61,0.005,0.012,Free,game_action,...,https://www.innersloth.com,support@innersloth.com,"Jun 15, 2018","Jun 14, 2021",Everyone 10+,https://www.innersloth.com/privacy.php,True,True,True,2021-06-15 21:57:23
4,4,9,Temple Run 2,9854530,500.0 M,4.46,0.002,0.005,Free,game_action,...,http://www.imangistudios.com/contact.html,support-tr2@imangistudios.com,"Jan 23, 2013","Jun 08, 2021",Everyone,http://imangistudios.com/privacy/privacy-polic...,True,True,True,2021-06-16 14:28:54


In [6]:
app_df2 = app_df[['title', 'App Id']]

In [7]:
app_df2.head()

,title,App Id
0,Mobile Legends: Bang Bang,com.mobile.legends
1,Brawl Stars,com.supercell.brawlstars
2,Shadow Fight 2,com.nekki.shadowfight
3,Among Us,com.innersloth.spacemafia
4,Temple Run 2,com.imangi.templerun2


In [17]:
app_df2['idx_no'] = range(0,5283)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
app_df2.head(10)

,title,App Id,idx_no
0,Mobile Legends: Bang Bang,com.mobile.legends,0
1,Brawl Stars,com.supercell.brawlstars,1
2,Shadow Fight 2,com.nekki.shadowfight,2
3,Among Us,com.innersloth.spacemafia,3
4,Temple Run 2,com.imangi.templerun2,4
5,PUBG MOBILE LITE,com.tencent.iglite,5
6,Standoff 2,com.axlebolt.standoff2,6
7,slither.io,air.com.hypah.io.slither,7
8,Gangstar Vegas: World of Crime,com.gameloft.android.ANMP.GloftGGHM,8
9,Talking Tom Gold Run,com.outfit7.talkingtomgoldrun,9


In [19]:
app_df2.tail(10)

,title,App Id,idx_no
5273,Millonario 2021 : Trivia Quiz Game,com.brainapp.spanishlanguagequizgame,5273
5274,毎日の耳かき(別),jp.co.liica.mimikaki2,5274
5275,Million Deal: Win A Million Dollars,com.niceteen.games.milliondeal,5275
5276,HiQ,com.hiride.social,5276
5277,左右TEMPO - 反應訓練遊戲,com.san.leftrighttempo,5277
5278,Deal The Big Deal,com.vmoga.dtbd,5278
5279,Milionerzy po polsku,pl.jmobile.milionerzy.po.polsku,5279
5280,Car Logos Quiz,com.hichamtachourit.carlogoquiz,5280
5281,من سيربح المليون,com.million.win.will.who.wwwm,5281
5282,Yo Nunca Argentino,com.emisferio.yonunca,5282


In [8]:
## Get list of app names and app IDs
app_names = list(app_df2['title'])
app_ids = list(app_df2['App Id'])

In [9]:
len(app_ids)

5283

In [46]:
app(app_ids[404])['released']

'Jun 23, 2014'

In [52]:
a=range(0, 5283)
len(a)

5283

In [ ]:
# # for문을 통해 현재 서비스 종료한 앱 확인
# app_info = []
# while True:
#   for i in range(0, 5283):
#     try:
#       app_info.append(app(app_ids[i])['released'])
#     except:
#       print(f'계속 진행해 {i}')
#       continue

계속 진행해 800
계속 진행해 817
계속 진행해 822
계속 진행해 825
계속 진행해 826
계속 진행해 829
계속 진행해 831
계속 진행해 833
계속 진행해 835
계속 진행해 836
계속 진행해 841


In [11]:
# 서비스 지속 확인
app(app_ids[0])['title']

'Mobile Legends: Bang Bang'

In [12]:
# # 서비스 종료 확인
# app(app_ids[60])['title']

NotFoundError: ignored

In [13]:
# # 서비스 종료 확인
# app(app_ids[146])['title']

NotFoundError: ignored

In [21]:
## 서비스 종료 앱 인덱스 가져오기
sv_end_df = pd.read_excel('/content/drive/MyDrive/Python_SNU/mobilegame_analysis/service_end_idx_2.xlsx')
sv_end_df.head()

,sv_end_idx,sv_end_stat
0,0,service_ongoing
1,1,service_ongoing
2,2,service_ongoing
3,3,service_ongoing
4,4,service_ongoing


In [22]:
sv_end_df.tail()

,sv_end_idx,sv_end_stat
5278,5278,service_ongoing
5279,5279,service_ongoing
5280,5280,service_ongoing
5281,5281,service_ongoing
5282,5282,service_ongoing


In [23]:
df_INNER_JOIN2 = pd.merge(app_df2, sv_end_df, left_on='idx_no', right_on='sv_end_idx', how='inner')

In [24]:
df_INNER_JOIN2.head()

,title,App Id,idx_no,sv_end_idx,sv_end_stat
0,Mobile Legends: Bang Bang,com.mobile.legends,0,0,service_ongoing
1,Brawl Stars,com.supercell.brawlstars,1,1,service_ongoing
2,Shadow Fight 2,com.nekki.shadowfight,2,2,service_ongoing
3,Among Us,com.innersloth.spacemafia,3,3,service_ongoing
4,Temple Run 2,com.imangi.templerun2,4,4,service_ongoing


In [25]:
df_INNER_JOIN2.tail()

,title,App Id,idx_no,sv_end_idx,sv_end_stat
5278,Deal The Big Deal,com.vmoga.dtbd,5278,5278,service_ongoing
5279,Milionerzy po polsku,pl.jmobile.milionerzy.po.polsku,5279,5279,service_ongoing
5280,Car Logos Quiz,com.hichamtachourit.carlogoquiz,5280,5280,service_ongoing
5281,من سيربح المليون,com.million.win.will.who.wwwm,5281,5281,service_ongoing
5282,Yo Nunca Argentino,com.emisferio.yonunca,5282,5282,service_ongoing
